<a href="https://colab.research.google.com/github/ichekhovskikh/cyberleninka-article-downloader/blob/master/cyberleninka_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install requests

import requests
import re
import json

Монтирование директории для работы с файлами

In [17]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Указание пути для сохранения корпусов

In [0]:
train_save_path = '/content/drive/My Drive/corpus/train_corpus.json'
test_save_path = '/content/drive/My Drive/corpus/test_corpus.json'

Указание тем и количество статей для обучения и тестирования

In [0]:
search_tags = ['машинное обучение', 'космические снимки', 'российская история', 'квантовая физика', 'молекулярная химия']
train_article_count = 100
test_article_count = 10

Укзание страниц сайта

In [0]:
search_url = 'https://cyberleninka.ru/api/search'
article_url = 'https://cyberleninka.ru'

Создание сессии

In [0]:
session = requests.Session()

Сделать запрос поиска

In [0]:
def search_articles(search_query, from_index, size):
    params = {'mode': "articles", 'q': search_query, 'size': size, 'from': from_index}
    response = session.post(search_url, data = json.dumps(params))
    return json.loads(response.content)['articles']

Получение ссылок на тексты научных статей

In [0]:
def articles2links(articles):
    links = []
    for article in articles:
        links.append(article['link'])
    return links

Получение текста научной статьи по ссылке

In [0]:
def link2text(acticle_link):
    response = session.get(article_url + acticle_link)
    html_page = response.content.decode()
    matches = re.findall(r"<p>(.+|\n+)</p>", html_page)
    text = ""
    for match in matches:
       text = text + "\n" + match.replace("<p>", "").replace("</p>", "")
    return text

Конвертирование text в json

In [0]:
def text2json(text_id, class_name, text):
    clear_text = text.replace("\n", " ").replace("\t", " ").replace("\r", " ").replace('"', "'").replace("\\", "/")
    return '{"id":' + str(text_id) + ',"text":"' + clear_text + '","tags":["' + str(text_id) + '"],"class_name":"' + class_name + '"}'

Сохранить список json объектов

In [0]:
def save_json_list(data_file, data_path):
    save_file = "[" + data_file
    if len(save_file) > 1:
        save_file = save_file[:-1]
    save_file += "]"
    with open(data_path, 'w') as save_path:
        save_path.write(save_file)


Получить все статьи

In [0]:
train_corpus_file = ""
test_corpus_file = ""

train_size = train_article_count * len(search_tags)
train_id = 0
test_id = train_size

for search_tag in search_tags:
    train_articles = search_articles(search_tag, 0, train_article_count)
    test_articles = search_articles(search_tag, train_article_count, test_article_count)
    train_links = articles2links(train_articles)
    test_links = articles2links(test_articles)

    for link in train_links:
        text = link2text(link)
        acticle_json = text2json(train_id, search_tag, text)
        train_corpus_file += acticle_json + ","
        train_id += 1

    for link in test_links:
        text = link2text(link)
        acticle_json = text2json(test_id, search_tag, text)
        test_corpus_file += acticle_json + ","
        test_id += 1

save_json_list(train_corpus_file, train_save_path)
save_json_list(test_corpus_file, test_save_path)